In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Data Disc. and prep.

In [ ]:
pip install pm4py

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mn
import pm4py
from scipy.stats.mstats import winsorize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
log = pm4py.read_xes('/content/drive/MyDrive/BPI Dataset/Road_Traffic_Fine_Management_Process (1).xes')
df = pm4py.convert_to_dataframe(log)
df

# Data preproc

In [ ]:
df

In [ ]:
df['concept:name'].unique()

In [ ]:
df['case:concept:name']

In [ ]:
sum(df['case:concept:name'] == "A1")

In [ ]:
df['notificationType'] = df['notificationType'].replace('NaN',np.nan)
df['notificationType'].unique()

In [ ]:
pd.unique(df['case:concept:name'])

In [ ]:
df['case:concept:name'].describe()

In [ ]:
pd.to_datetime(df['time:timestamp'],utc=True)

In [ ]:
#find missings
mn.matrix(df)

In [ ]:
#Vehicleclass->Create Fine
df_vehicleclass = df[df['concept:name'] == "Create Fine"]
sns.countplot(x ='vehicleClass', data=df_vehicleclass)

In [ ]:
#find dismidal missings
df['dismissal'].unique()
df.dismissal.isnull()
x = df.loc[~df.dismissal.isnull()]
x['concept:name'].unique()
df_dismissal = df[df['concept:name'] == ("Create Fine" or "Send Appeal to Prefecture" or "Appeal to Judge")]
df_dismissal
mn.matrix(df_dismissal)

In [ ]:
#find article missings
df['article'].unique()
df.article.isnull()
x = df.loc[~df.article.isnull()]
x['concept:name'].unique()
dfarticle = df[df['concept:name'] == ("Create Fine")]
mn.matrix(dfarticle)

In [ ]:
def findactivity(k):
  x = df.loc[~df[k].isnull()]
  f = x['concept:name'].unique()
  h = list(map(lambda x:x,f))
  return h
def showactivity(x):
  dfx = df[df['concept:name'] == (x)]
  return mn.matrix(dfx)

In [ ]:
df.head()

In [ ]:
findactivity("amount")

In [ ]:
showactivity('Create Fine'or 'Add penalty')

# Finding Outliers

In [ ]:
findactivity("amount")
sns.boxplot(x="amount",data=df[(df['concept:name'] == ('Create Fine' or 'Add penalty'))])

In [ ]:
#Outer fences of the variable CRIM
def fences(df, variable_name):    
    q1 = df[variable_name].quantile(0.25)
    q3 = df[variable_name].quantile(0.75)
    iqr = q3-q1
    outer_fence = 3*iqr
    outer_fence_le = q1-outer_fence
    outer_fence_ue = q3+outer_fence
    return outer_fence_le, outer_fence_ue

outer_fence_le, outer_fence_ue = fences(df[df['concept:name'] == ('Create Fine' or 'Add penalty')], 'amount')
print('Lower end outer fence: ', outer_fence_le)
print('Upper end outer fence: ', outer_fence_ue)

In [ ]:
xd = df[df['concept:name'] == ('Create Fine' or 'Add penalty')]

In [ ]:
xd['amount'] = winsorize(xd['amount'].values, limits = [0, 0.09]).data

In [ ]:
sns.boxplot(x='amount',data = xd)

In [ ]:
sns.displot(xd, x="amount",bins = 10)
sns.displot(df, x="amount",bins = 5)

In [ ]:
findactivity("notificationType")
showactivity('Insert Fine Notification')

In [ ]:
findactivity("expense")

In [ ]:
xf = df[df['concept:name'] == ('Send Fine')]
xfa = df[df['concept:name'] == ('Send Fine')]
xfa2 = df[df['concept:name'] == ('Send Fine')]

In [ ]:
fences(xf,'expense')

In [ ]:
#sns.boxplot(x='expense',data = xf)
fences(xf,'expense')
sns.boxplot(x='expense',data = xf[xf['expense']<24.0])

In [ ]:
zzz = df['case:concept:name'].unique()
len(zzz)

In [ ]:
xfa2 = xf[xf['expense']>24]
zz = xfa2['case:concept:name'].unique()
len(zz)

In [ ]:
xfa['expense'] = winsorize(xf['expense'].values, limits = [0, 0.09]).data
sns.boxplot(x='expense',data = xfa)

In [ ]:
findactivity("article")
xa = df[df['concept:name'] == ('Create Fine')]
xa.article = xa.article.astype(str)
uxa =xa.article.unique()
plt.figure(figsize=(50,8))
sns.countplot(x="article",data=xa[xa["article"]!=('7.0' and '157.0' and '158.0')])



# Data dic

In [ ]:
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
# process mining 
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
# viz
from pm4py.visualization.petrinet import visualizer as pn_visualizer
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer
from pm4py.visualization.dfg import visualizer as dfg_visualization
# misc 
from pm4py.objects.conversion.process_tree import converter as pt_converter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from pm4py.algo.filtering.log.start_activities import start_activities_filter
from pm4py.algo.filtering.log.end_activities import end_activities_filter
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.log import case_statistics

In [ ]:
variants = variants_filter.get_variants(log)  
print(f"We have:{len(variants)} variants in our log")

In [ ]:
variants_count = case_statistics.get_variant_statistics(log)
variants_count = sorted(variants_count, key=lambda x: x['count'], reverse=True)
variants_count

In [ ]:
from pm4py.algo.filtering.log.attributes import attributes_filter
activities = attributes_filter.get_attribute_values(log, "concept:name")
activities
max(activities, key=activities.get)

In [ ]:
activities2 = attributes_filter.get_attribute_values(log, "case:concept:name")
activities2
max(activities2, key=activities2.get)

In [ ]:
sum(activities2.values()) / len(activities2)

3.7339229899581032

# PM TO DM :)

In [ ]:
ca = df['case:concept:name'].unique()
l = {}
for i in ca:
  l[i] = []
  
for i,j in zip(df['case:concept:name'],df['concept:name']):
   l[i].append(j)

In [ ]:
df2 = pd.DataFrame()

In [ ]:
xfg = df['concept:name'].unique()
lam = {}
for i in xfg:
  lam[i] = {}

df4 = pd.DataFrame(lam)

In [ ]:
lfp = {}
for i in df:
  lfp[i] = {}
df5 = pd.DataFrame(lfp)

In [ ]:
df2['caseid'] = df['case:concept:name'].unique()
df3 = pd.concat([df2,df5,df4],axis=1)

In [ ]:
dx_dic = {}
for col in df3.columns:
    dx_dic[col] = df3.columns.get_loc(col)

The code below needs a lot of runtime so we cant easily get what we want from it

In [ ]:
for i in df3['caseid']:
  lis = list(df['concept:name'].unique())
  for j in l[i]:
    v = int(np.where(df3["caseid"] == i)[0])
    b = dx_dic[j]
    df3.iloc[v,b] = 1
    try:
      lis.remove(j)
    except:
      pass
  for j in lis:
    b = dx_dic[j]
    df3.iloc[v,b]  = 0  


so what we do is to split the dataframe into 10 pieces and then compile all of them

In [ ]:
z = [1,2,3,4,5,6,7,8,9,10]
for i in z:
  if i == 1 :
    k = int(df3.shape[0]/10)
    exec(f'dft{i} = df3.iloc[:k,]')
  else:
    k = int(df3.shape[0]/10 * (i-1))
    f = int(df3.shape[0]/10 * i)
    exec(f'dft{i} = df3.iloc[k:f,]')

In [ ]:
dft_samp = []
z = [1,2,3,4,5,6,7,8,9,10]
z_new = [6,7,8,9,10]
for i in z_new: 
  dft_samp.append(f'dft{i}')

In [ ]:
"dft{0}".format(i)

In [ ]:
dx_dic = {}
for col in df3.columns:
    dx_dic[col] = df3.columns.get_loc(col)

In [ ]:
for h in dft_samp :
  i_df = globals()[h]
  for i in i_df['caseid']:
    lis = list(df['concept:name'].unique())
    for j in l[i]:
        v = int(np.where(i_df["caseid"] == i)[0])
        b = dx_dic[j]
        i_df.iloc[v,b] = 1
        try:
          lis.remove(j)
        except:
            pass
    for j in lis:
      b = dx_dic[j]
      i_df.iloc[v,b]  = 0
  i_df.to_csv("/content/drive/MyDrive/BPI Dataset/DFS/"+str(h)+".csv")  

Now we concat them together again.

In [ ]:
dfh_samp = []
for i in z: 
  dfh_samp.append(f'dfh{i}')
for i in z:
  exec(f'dfh{i} = pd.read_csv("/content/drive/MyDrive/BPI Dataset/DFS/dft"+str(i)+".csv")')

In [ ]:
dforg = pd.concat([dfh1,dfh2,dfh3,dfh4,dfh5,dfh6,dfh7,dfh8,dfh9,dfh10])

In [ ]:
dforg.to_csv("/content/drive/MyDrive/BPI Dataset/DFS/dforg.csv")

In [16]:
dforg = pd.read_csv("/content/drive/MyDrive/BPI Dataset/DFS/dforg.csv")
dforg = dforg.set_index('index')

In [ ]:
dforg = dforg.set_index('index')

In [ ]:
dforg["index"] = dforg["Unnamed: 0"]

In [ ]:
dforg = dforg.drop(['Unnamed: 0', 'lifecycle:transition' , 'case:concept:name' , 'amount', 'concept:name'], axis=1)

Now lets go do the logic for the rest of the columns.

In [ ]:
dforg.head()

In [18]:
dforg.rename(columns = {'time:timestamp' : 'duration(days)'}, inplace = True)

In [ ]:
dic_for_res = {}

for i in ca:
  dic_for_res[i] = []

In [ ]:
from numpy import nan

In [ ]:
for i,j in zip(df['case:concept:name'],df['org:resource']):
  if pd.isnull(j) == False :
    dic_for_res[i].append(j)

In [ ]:
xvb = df['org:resource'].dropna().unique().tolist()

In [ ]:
dx_dic = {}
for col in dforg:
    dx_dic[col] = dforg.columns.get_loc(col)

In [ ]:
dx_dic["org:resource"]

In [ ]:
for i in dforg['caseid']:
  for j in dic_for_res[i]:
    v = int(np.where(dforg["caseid"] == i)[0])
    dforg.iloc[v,1] = j

In [ ]:
dforg

In [ ]:
df['dismissal'].unique()

array(['NIL', nan, '#', 'G', 'N', 'K', '5', '3', 'A', 'I', 'D', 'T', 'E',
       '@', 'M', 'Q', 'F', 'V', 'U', 'C', 'B', '$', 'Z', 'J', 'R', '2',
       '4'], dtype=object)

In [ ]:
dismis_uniq = df['dismissal'].unique()
list(dismis_uniq)

In [ ]:
dismis_uniq = list(dismis_uniq)

In [ ]:
caseunique = list(df['case:concept:name'].unique())

In [ ]:
l = {}
for i,j in zip(df['case:concept:name'],df['dismissal']) :
    if pd.isnull(j) == False :
      l[i] = j

In [ ]:
df[df['dismissal'] == "M"]

In [ ]:
l["N23119"]

In [ ]:
for i in dforg['caseid']:
  v = int(np.where(dforg["caseid"] == i)[0])
  dforg.iloc[v,2] = l[i]

In [ ]:
l = {}
for i,j in zip(df['case:concept:name'],df['totalPaymentAmount']):
  if pd.isnull(j) == False:
    l[i] = j

In [ ]:
for i in dforg['caseid']:
  v = int(np.where(dforg["caseid"] == i)[0])
  dforg.iloc[v,4] = l[i]

In [ ]:
#df[df['vehicleClass'].isnull() == True]
findactivity("vehicleClass")

['Create Fine']

In [ ]:
showactivity("Create Fine")

In [ ]:
l = {}
for i,j in zip(df['case:concept:name'],df['vehicleClass']):
  if pd.isnull(j) == False:
    l[i] = j

In [ ]:
l["A100"]

'A'

In [ ]:
np.unique(list(l.values()))

In [ ]:
for i in caseunique:
  v = int(np.where(dforg["caseid"] == i) [0])
  dforg.iloc[v,3] = l[i]

Now lets get every case duration time so we fill our next column ;)

In [ ]:
from pm4py.statistics.traces.generic.log import case_statistics
median_case_duration = case_statistics.get_median_caseduration(log, parameters={
    case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"
})

In [ ]:
from pm4py.statistics.traces.generic.log import case_statistics
all_case_durations = case_statistics.get_all_casedurations(log, parameters={
    case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"})

In [ ]:
p = []
for i in all_case_durations:
  if i == 0 :
    pass
  else:
    p.append(i)


In [ ]:
import math
all_case_duration_hour = []
for i in all_case_durations:
  min = i/60
  hour = min/60
  day = hour/24
  value = math.ceil(day)
  all_case_duration_hour.append(value)

In [ ]:
case_arrival_ratio = pm4py.get_case_arrival_average(log)

In [ ]:
df

In [ ]:
day_dic = {}
countss = []
caseunique = list(df['case:concept:name'].unique())
for i in caseunique:
  dfk = df[df['case:concept:name'] == i]
  dfk.sort_values(by="time:timestamp")
  row_num = dfk.shape[0]
  duration = dfk.iloc[(row_num - 1),7] - dfk.iloc[0,7]
  durationday = duration.days
  day_dic[i] = durationday
  countss.append(i)

In [ ]:
c = df.iloc[1,7] - df.iloc[0,7]
c.days

In [ ]:
len(day_dic)

In [40]:
def replaceinnew (y):
  columns = dforg.columns
  l = {}
  for i,j in zip(df['case:concept:name'],df[y]):
    if pd.isnull(j) == False:
      l[i] = j
  for i in l:
    v = int(np.where(dforg["caseid"] == i)[0])
    m = int(columns.get_loc(y))
    dforg.iloc[v,m] = l[i]
  dforg.to_csv("/content/drive/MyDrive/BPI Dataset/DFS/dforg.csv")

In [41]:
replaceinnew('article')

In [45]:
replaceinnew('points')

In [46]:
replaceinnew('notificationType')

In [49]:
replaceinnew('matricola')

In [50]:
dforg = dforg.drop(['matricola', 'lastSent' , 'paymentAmount'], axis=1)

In [52]:
dforg = dforg.drop(['points'], axis=1)

In [ ]:
dforg

In [53]:
columns = dforg.columns
l = {}
for i,j in zip(df['case:concept:name'],df['expense']):
  if pd.isnull(j) == False:
    l[i] = []
    l[i].append(j)
for i in l:
  v = int(np.where(dforg["caseid"] == i)[0])
  m = int(columns.get_loc('expense'))
  dforg.iloc[v,m] = sum(l[i])
dforg.to_csv("/content/drive/MyDrive/BPI Dataset/DFS/dforg.csv")

In [ ]:
caseunique = list(df['case:concept:name'].unique())

In [ ]:
for i in caseunique:
  v = int(np.where(dforg["caseid"] == i)[0])
  dforg.iloc[v,5] = day_dic[i]

In [ ]:
from pm4py.statistics.traces.generic.log import case_arrival
case_arrival_ratio = case_arrival.get_case_arrival_avg(log, parameters={
    case_arrival.Parameters.TIMESTAMP_KEY: "time:timestamp"})

In [ ]:
## Import the alpha_miner algorithm
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
net, initial_marking, final_marking = alpha_miner.apply(log)
## Import the petrinet visualizer object
from pm4py.visualization.petrinet import visualizer as pn_visualizer
# Visualise 
gviz = pn_visualizer.apply(net, initial_marking, final_marking) 
pn_visualizer.view(gviz)

In [ ]:
gviz = pn_visualizer.apply(net, initial_marking, final_marking,variant=pn_visualizer.Variants.FREQUENCY, log=log)
pn_visualizer.view(gviz)

In [ ]:
from pm4py.algo.filtering.log.attributes import attributes_filter

x, y = attributes_filter.get_kde_date_attribute(log, attribute="time:timestamp")

from pm4py.visualization.graphs import visualizer as graphs_visualizer

gviz = graphs_visualizer.apply_plot(x, y, variant=graphs_visualizer.Variants.DATES)
graphs_visualizer.view(gviz)

In [ ]:
from pm4py.statistics.eventually_follows.log import get as efg_get
efg_graph = efg_get.apply(log)
print(efg_graph)

In [ ]:
df["concept:name"].unique()

In [ ]:
pm4py.view_performance_spectrum(log, ["Create Fine" , "Send Fine" , "Insert Fine Notification", "Send for Credit Collection"], format="png")

# Machine Learning